In [336]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [337]:
timeframe = 40
stride = 1
num_features = 5

In [338]:
x_train = np.loadtxt('./data/X_train_transformed.csv')
x_train = x_train.reshape(x_train.shape[0], timeframe, num_features)
y_train = np.loadtxt('./data/y_train_transformed.csv').astype(int)
x_test = np.loadtxt('./data/X_test_transformed.csv')
x_test = x_test.reshape(x_test.shape[0], timeframe, num_features)
y_test = np.loadtxt('./data/y_test_transformed.csv').astype(int)

In [339]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((21560, 40, 5), (21560,), (5361, 40, 5), (5361,))

In [340]:
n_classes = 3

In [341]:
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(timeframe, num_features)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(n_classes, activation='softmax')
])

/home/bina/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [342]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [343]:
print(model.summary())

Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_21 (Conv1D)              │ (None, 39, 64)         │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_21 (MaxPooling1D) │ (None, 19, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ (None, 1216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 64)             │        77,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,787 (307.76 KB)

 Trainable params: 78,787 (307.76 KB)

 Non-trainable params: 0 (0.00 B)

None


In [344]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [345]:
unique_labels = np.unique(y_train)
class_mapping = {label: i for i, label in enumerate(unique_labels)}
y_train_encoded = np.array([class_mapping[label] for label in y_train])
y_test_encoded = np.array([class_mapping[label] for label in y_test])

In [346]:
history = model.fit(x_train, y_train_encoded, epochs=50, batch_size=32, validation_data=(x_test, y_test_encoded), callbacks=[early_stopping])

Epoch 1/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5934 - loss: 0.8650 - val_accuracy: 0.5917 - val_loss: 0.8615
Epoch 2/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5897 - loss: 0.8595 - val_accuracy: 0.5917 - val_loss: 0.8582
Epoch 3/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5923 - loss: 0.8565 - val_accuracy: 0.5917 - val_loss: 0.8609
Epoch 4/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5942 - loss: 0.8586 - val_accuracy: 0.5917 - val_loss: 0.8591
Epoch 5/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5969 - loss: 0.8554 - val_accuracy: 0.5917 - val_loss: 0.8605
Epoch 6/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5930 - loss: 0.8559 - val_accuracy: 0.5917 - val_loss: 0.8611
Epoch 7/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5958 - loss: 0.8533 - val_accuracy: 0.5917 - val_loss: 0.8611


In [348]:
loss, accuracy = model.evaluate(x_test, y_test_encoded)
print("Loss:", loss)
print("Accuracy:", accuracy)

168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - accuracy: 0.5915 - loss: 0.8649
Loss: 0.8581851124763489
Accuracy: 0.591680645942688
